# Vector Store Creation with SQL

In [1]:
import teradataml as tdml
import json

## 1 - Connect to Vantage

In [2]:
print('teradataml version    :',tdml.__version__)

teradataml version    : 20.00.00.05


In [ ]:
# Reading JSON data
with open('../connection/connection.json', 'r') as f:
    Param = json.load(f)
    Param['database'] = Param['user']
                
tdml.create_context(**Param)

## 2 - The data

In [4]:
table_name = 'denis_pdfs'
database   = Param['database']

In [5]:
print(tdml.execute_sql(f"SHOW TABLE {database}.{table_name}").fetchall()[0][0].replace('\r','\n'))

CREATE SET TABLE dm250067.denis_pdfs ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP2
     (
      TD_ID INTEGER,
      TD_FILENAME VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC,
      CHUNKS VARCHAR(32000) CHARACTER SET LATIN NOT CASESPECIFIC)
PRIMARY INDEX ( TD_ID );


In [12]:
dataset = tdml.DataFrame(tdml.in_schema(database,table_name))
dataset

TD_ID,TD_FILENAME,CHUNKS
40,2404.16130v1.pdf,"which people inspect, engage with, and contextualize data within the broader scope of real-world activities (Koesten et al., 2021). Similarly, methods for extracting latent summarization queries from source texts also exist (Xu and Lapata, 2021), but such extracted questions can target details that betray prior knowledge of the texts. To evaluate the effectiveness of RAG systems for more global sensemaking tasks, we need questions that convey only a high-level understanding of dataset contents,"
120,2404.03622v3.pdf,"ning empty squares is : B. 3 The final configuration would look like this: Visualize the state after each reasoning step Lets think step by step. (a) Common behaviors of V oT and CoT prompting in visual tiling task, with the overall track rate of 87.1% and 57.4% respectively. To navigate from to , we need to find the path that avoids obstacles ( ) and follows the roads ( ). Here's the step -by-step navigation : 1. Move down from to the end of the continuous road. After the move: 2."
57,2404.03622v3.pdf,ing Rate Succ Rate GPT-3.5 CoT 16.10 2.62 17.42 44.10 8.50 GPT-3.5 V oT 19.02 1.61 13.10 47.99 9.00 LLAMA3-8B CoT 4.65 0 28.73 47.24 16.50 LLAMA3-8B V oT 4.97 0.2 26.75 46.73 15.50 LLAMA3-70B CoT 19.90 2.62 49.01 56.41 26.00 LLAMA3-70B V oT 30.24 5.85 54.09 56.03 32.50 Table 3: Performance of V oT in GPT-3.5 and LLAMA3 models. Underline denotes statistical significance with p < 0.05 compared to corresponding CoT baseline using two-sample z-test. geometric patterns [ Cho19 ] and 3D spatial inform
57,2404.07143v2.pdf,"methods such as position interpolation techniques (Chen et al., 2023a) can be data efficient as they only adjust the positional bias in the attention layer, they are still costly for inference. The attention mechanism is also prone to the issues of attention sink (Xiao et al., 2023) and lost-in-the-middle (Liu et al., 2024). Consequently, they struggle in a regime where context length is longer than what was observed during training (Press et al., 2021; Kazemnejad et al., 2024). The proposed In"
57,Efficiency-Productivity-and-Speed-to-Deployment-MD007074.pdf,"n Enterprise Feature Store must be actively managed. Due consideration must be given not only to data freshness and the frequency of updates, but also to the removal of redundant features that are no longer being used. Precise cataloguing of all features is critical to prevent a feature store becoming a feature swamp. The dependencies between and within features (including data, processes, models and functions) must be clearly documented and understood. Users must be able to discover an"
158,2404.03622v3.pdf,"Tetromino L fits perfectly in the top left corner. Now, we can place the Tetromino I in the remaining space: ``` ``` The Tetromino I fits perfectly in the remaining space using Variation 1 (vertical placement). Therefore, the correct variation of Tetromino L that fits into the target rectangle is: A. 1 The initial attempt to place Variation 1 of Tetromino L was incorrect because it was not placed in the top left corner. Upon correcting the placement, we find that Variation 1"
57,2404.16130v1.pdf,"prehensive and structured overview of public figures across various sectors of the entertainment industry, including film, television, music, sports, and digital media. It lists multiple individuals, providing specific examples of their contributions and the context in which they are mentioned in entertainment articles, along with references to data reports for each claim. This approach helps the reader understand the breadth of the topic and make informed judgments without being misled. In cont"
40,Efficiency-Productivity-and-Speed-to-Deployment-MD007074.pdf,"al organization will determine what analysis is planned, what system resources are available, and how to define the refresh methodology. Obviously, it is not possible to compute every possible variable in an EFS and models will invariably have feature

## 3 - Vector Embedding Process : Using AWS Bedrock

### 3.1 - The use of td_byone()

Using AWS Bedrock requires a single AMP operation. For that, we will use the function td_byone()

In [15]:
query_dataset = f"""
SELECT
{'\n, '.join([x for x in dataset.columns])}
, td_byone() AS  FOR_SINGLE_AMP_ATTRIBUTION
FROM {database}.{table_name}
"""

print(query_dataset)


SELECT
TD_ID
, TD_FILENAME
, CHUNKS
, td_byone() AS  FOR_SINGLE_AMP_ATTRIBUTION
FROM dm250067.denis_pdfs



In [16]:
tdml.DataFrame.from_query(query_dataset)

TD_ID,TD_FILENAME,CHUNKS,FOR_SINGLE_AMP_ATTRIBUTION
40,2404.16130v1.pdf,"which people inspect, engage with, and contextualize data within the broader scope of real-world activities (Koesten et al., 2021). Similarly, methods for extracting latent summarization queries from source texts also exist (Xu and Lapata, 2021), but such extracted questions can target details that betray prior knowledge of the texts. To evaluate the effectiveness of RAG systems for more global sensemaking tasks, we need questions that convey only a high-level understanding of dataset contents,",26535
120,2404.03622v3.pdf,"ning empty squares is : B. 3 The final configuration would look like this: Visualize the state after each reasoning step Lets think step by step. (a) Common behaviors of V oT and CoT prompting in visual tiling task, with the overall track rate of 87.1% and 57.4% respectively. To navigate from to , we need to find the path that avoids obstacles ( ) and follows the roads ( ). Here's the step -by-step navigation : 1. Move down from to the end of the continuous road. After the move: 2.",26535
57,2404.03622v3.pdf,ing Rate Succ Rate GPT-3.5 CoT 16.10 2.62 17.42 44.10 8.50 GPT-3.5 V oT 19.02 1.61 13.10 47.99 9.00 LLAMA3-8B CoT 4.65 0 28.73 47.24 16.50 LLAMA3-8B V oT 4.97 0.2 26.75 46.73 15.50 LLAMA3-70B CoT 19.90 2.62 49.01 56.41 26.00 LLAMA3-70B V oT 30.24 5.85 54.09 56.03 32.50 Table 3: Performance of V oT in GPT-3.5 and LLAMA3 models. Underline denotes statistical significance with p < 0.05 compared to corresponding CoT baseline using two-sample z-test. geometric patterns [ Cho19 ] and 3D spatial inform,26535
57,2404.07143v2.pdf,"methods such as position interpolation techniques (Chen et al., 2023a) can be data efficient as they only adjust the positional bias in the attention layer, they are still costly for inference. The attention mechanism is also prone to the issues of attention sink (Xiao et al., 2023) and lost-in-the-middle (Liu et al., 2024). Consequently, they struggle in a regime where context length is longer than what was observed during training (Press et al., 2021; Kazemnejad et al., 2024). The proposed In",26535
57,Efficiency-Productivity-and-Speed-to-Deployment-MD007074.pdf,"n Enterprise Feature Store must be actively managed. Due consideration must be given not only to data freshness and the frequency of updates, but also to the removal of redundant features that are no longer being used. Precise cataloguing of all features is critical to prevent a feature store becoming a feature swamp. The dependencies between and within features (including data, processes, models and functions) must be clearly documented and understood. Users must be able to discover an",26535
158,2404.03622v3.pdf,"Tetromino L fits perfectly in the top left corner. Now, we can place the Tetromino I in the remaining space: ``` ``` The Tetromino I fits perfectly in the remaining space using Variation 1 (vertical placement). Therefore, the correct variation of Tetromino L that fits into the target rectangle is: A. 1 The initial attempt to place Variation 1 of Tetromino L was incorrect because it was not placed in the top left corner. Upon correcting the placement, we find that Variation 1",26535
57,2404.16130v1.pdf,"prehensive and structured overview of public figures across various sectors of the entertainment industry, including film, television, music, sports, and digital media. It lists multiple individuals, providing specific examples of their contributions and the context in which they are mentioned in entertainment articles, along with references to data reports for each claim. This approach helps the reader understand the breadth of the topic and make informed judgments without being misled. In cont",26535
40,Efficiency-Productivity-and-Speed-to-Deployment-MD007074.pdf,"al organization will determine what analysis is planned, what system resources are available, and how to define the refresh methodology. Obviously, it is not possible to compute ever

### 3.2 - Creation of the Embedding with AI_TEXTEMBEDDINGS

In [17]:
vector_store_table   = 'denis_pdfs_embeddings'
primary_index        = ['TD_ID']
text_column          = 'chunks'

embedding_model      = 'amazon.titan-embed-text-v1'
apitype              = 'aws'
region               = 'us-west-2'
authorization_object = 'AWSEmbeddingsAuth'

In [18]:
tdml.execute_sql(f"DROP TABLE {database}.{vector_store_table}")

TeradataCursor uRowsHandle=68 bClosed=False

In [19]:
embedding_query = f"""
CREATE MULTISET TABLE {database}.{vector_store_table} AS
(
SELECT *
FROM AI_TEXTEMBEDDINGS (
    ON ({'\n\t'.join(query_dataset.split('\n'))}) AS InputTable
    PARTITION BY FOR_SINGLE_AMP_ATTRIBUTION
    USING
        authorization({authorization_object})
        TextColumn('{text_column}')
        ApiType('{apitype}')
        REGION('{region}')
        ModelName('{embedding_model}')
        outputformat('vector')
        Accumulate({",".join(["'"+x+"'" for x in dataset.columns if x != text_column])})
) AS DT
) WITH DATA
PRIMARY INDEX ({','.join(primary_index)})
"""

print(embedding_query)


CREATE MULTISET TABLE dm250067.denis_pdfs_embeddings AS
(
SELECT *
FROM AI_TEXTEMBEDDINGS (
    ON (
	SELECT
	TD_ID
	, TD_FILENAME
	, CHUNKS
	, td_byone() AS  FOR_SINGLE_AMP_ATTRIBUTION
	FROM dm250067.denis_pdfs
	) AS InputTable
    PARTITION BY FOR_SINGLE_AMP_ATTRIBUTION
    USING
        authorization(AWSEmbeddingsAuth)
        TextColumn('chunks')
        ApiType('aws')
        REGION('us-west-2')
        ModelName('amazon.titan-embed-text-v1')
        outputformat('vector')
        Accumulate('TD_ID','TD_FILENAME','CHUNKS')
) AS DT
) WITH DATA
PRIMARY INDEX (TD_ID)



In [20]:
%%time
tdml.execute_sql(embedding_query)

CPU times: total: 15.6 ms
Wall time: 41 s


TeradataCursor uRowsHandle=69 bClosed=False

In [21]:
vector_table = tdml.DataFrame(tdml.in_schema(database, vector_store_table))
vector_table

TD_ID,TD_FILENAME,CHUNKS,Embedding,Message
95,2404.16130v1.pdf,"h/. Laskar, M. T. R., Hoque, E., and Huang, J. (2020). Query focused abstractive summarization via incorporating query relevance and transfer learning with transformer models. In Advances in Artificial Intelligence: 33rd Canadian Conference on Artificial Intelligence, Canadian AI 2020, Ottawa, ON, Canada, May 1315, 2020, Proceedings 33 , pages 342348. Springer. Laskar, M. T. R., Hoque, E., and Huang, J. X. (2022). Domain adaptation with pre-trained transform- ers for query-focused abstractive te","0.65625,-0.116211,0.119629,-0.214844,0.0476074,-0.220703,0.0708008,-9.01222e-05,-0.00836182,0.0693359,0.137695,0.695312,-0.0500488,0.179688,-0.316406,0.314453,0.229492,0.416016,-0.925781,0.0366211,-0.165039,-0.137695,-0.206055,0.351562,-0.132812,0.196289,0.104004,0.585938,-0.458984,0.00166321,0.314453,0.065918,0.546875,0.300781,-0.287109,0.121582,0.515625,0.367188,-0.12207,-0.0874023,0.769531,-0.135742,0.111328,-0.0786133,0.0952148,-0.00805664,-0.125,0.0922852,-0.78125,0.0849609,-0.134766,-0.271484,0.239258,0.328125,-0.265625,-0.125977,-0.0510254,-0.0620117,0.0678711,0.330078,0.100586,0.359375,0.078125,0.199219,-0.188477,0.269531,0.28125,-0.0512695,0.550781,0.273438,-0.192383,-0.0108032,0.378906,0.265625,1.14844,-0.125,-0.376953,0.585938,0.111328,-0.490234,-0.585938,0.558594,0.0756836,0.466797,0.139648,0.206055,-0.447266,0.0554199,0.000127792,0.382812,0.335938,0.176758,0.355469,-0.259766,0.447266,-0.198242,-0.0177002,0.175781,0.0966797,-0.515625,0.0568848,-0.339844,0.100098,-0.0678711,0.0834961,-0.214844,-0.0",
112,2404.03622v3.pdf,"ou made following movements: 1. Move right to the end of continuous road. 2. Move down to the end of continuous road. 3. Move left to the end of continuous road. What's the direction of next movement? A. Up B. Left C. Down D. Right Visualize the state after each reasoning step.Navigation Task: for a provided map, is the home as starting point, is the office as the destination. means the road, means the obstacle. There exists one and only one viable route for each map. Each ste","0.792969,-0.0673828,0.255859,0.133789,0.648438,0.224609,0.0476074,0.000133514,-0.251953,0.345703,0.703125,-0.462891,0.384766,0.396484,-0.0576172,-0.234375,0.384766,0.613281,-0.5625,0.298828,-0.753906,0.265625,-0.0917969,-0.100098,0.115234,0.195312,0.65625,-0.488281,0.0234375,-0.365234,-0.546875,0.333984,0.216797,0.46875,-0.691406,-0.519531,0.246094,-0.097168,0.425781,-0.124512,-0.851562,0.59375,-0.198242,0.318359,-0.396484,0.208008,0.0927734,-0.78125,0.53125,0.353516,0.00750732,0.0576172,-0.423828,-0.392578,-0.200195,0.3125,0.279297,-0.376953,-0.0546875,-0.209961,-0.410156,0.613281,0.118652,-0.408203,-0.140625,0.273438,0.546875,-0.378906,0.0742188,0.496094,0.114258,-0.554688,0.292969,0.103516,0.628906,0.294922,0.0446777,0.114746,0.0344238,0.378906,0.257812,-0.211914,-0.546875,0.28125,-0.251953,0.765625,0.458984,0.167969,0.000137329,0.00891113,-0.34375,0.316406,0.019165,-0.339844,0.0610352,0.0322266,-0.310547,0.081543,0.578125,-0.458984,-0.388672,0.320312,0.090332,0.225586,-0.480469,0.478516,0.205078,0.0844727",
49,2404.03622v3.pdf,"sured by the proportion of correct answers when the corresponding visualization is generated accurately. As could be seen from Table 2, LLMs demonstrate promising potential in performing multi-hop visualization while adhering to spatial constraints, with compliance rates of approximately 51-52%. However, the relatively low accuracy of state visualization (around 24%-26%) indicates a need for significant improvements in this area. Despite this limitation, LLMs are able to make correct decisions i","0.8125,-0.152344,0.439453,-0.419922,-0.109375,-0.142578,-0.15625,0.000246048,-0.180664,0.111328,0.353516,-0.10498,0.227539,0.0429688,-0.351562,0.00601196,0.15625,0.585938,-0.318359,0.186523,-0.11084,0.277344,0.105957,-0.115723,0.210938,-0.0678711,0.168945,0.429688,-0.104004,-0.0144653,-0.140625,0.695312,0.337891,-0.09375,-0.230469,-0.0284424,-0.10

In [22]:
vector_table.columns

['TD_ID', 'TD_FILENAME', 'CHUNKS', 'Embedding', 'Message']

In [23]:
print(tdml.execute_sql(f"SHOW TABLE {database}.{vector_store_table}").fetchall()[0][0].replace('\r','\n'))

CREATE MULTISET TABLE dm250067.denis_pdfs_embeddings ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP2
     (
      TD_ID INTEGER,
      TD_FILENAME VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC,
      CHUNKS VARCHAR(32000) CHARACTER SET LATIN NOT CASESPECIFIC,
      Embedding SYSUDTLIB.Vector,
      Message VARCHAR(32000) CHARACTER SET UNICODE NOT CASESPECIFIC)
PRIMARY INDEX ( TD_ID );


### 3.3 - Vector Normalization: TD_VECTORNORMALIZE

In [25]:
vector_column            = 'Embedding'
idcolumns                = ['TD_ID']
embedding_dimension      = 1536
accumulate_columns       = ['TD_FILENAME','Message']

In [26]:
query_normalization = f"""
SELECT *
FROM TD_Vectornormalize(
    ON {database}.{vector_store_table} AS InputTable
    USING
        IDColumns( {','.join(["'"+x+"'" for x in idcolumns])})
        TargetColumns('{vector_column}')
        Accumulate({','.join(["'"+x+"'" for x in accumulate_columns])})
        Approach('UNITVECTOR')
        EmbeddingSize({embedding_dimension})    
) as DT
"""

print(query_normalization)


SELECT *
FROM TD_Vectornormalize(
    ON dm250067.denis_pdfs_embeddings AS InputTable
    USING
        IDColumns( 'TD_ID')
        TargetColumns('Embedding')
        Accumulate('TD_FILENAME','Message')
        Approach('UNITVECTOR')
        EmbeddingSize(1536)    
) as DT



In [ ]:
tdml.DataFrame.from_query(query_normalization)